# Audible Data

## Import & Data Call

In [1]:
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException
from webdriver_manager.chrome import ChromeDriverManager
import re
import numpy as np

In [2]:
df = pd.read_parquet("./Data/Raw/audible_uncleaned.parquet")

## Webscraping USD to INR Exchange Rate

In [3]:
desired = 'USD'
origin = 'INR'
weblink = f'https://www.xe.com/currencyconverter/convert/?Amount=1&From={origin}&To={desired}'
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get(weblink)
soup = BeautifulSoup(driver.page_source, 'html.parser')
raw_text = soup.find('p', {'class': 'sc-423c2a5f-1 gPUWGS'}).text.strip()
conversion_rate = float(re.findall(r'\d+\.\d+', raw_text)[0])
driver.quit()

In [4]:
df.head(3)

,name,author,narrator,time,releasedate,language,stars,price
0,Geronimo Stilton #11 & #12,Writtenby:GeronimoStilton,Narratedby:BillLobely,2 hrs and 20 mins,04-08-08,English,5 out of 5 stars34 ratings,468.00
1,The Burning Maze,Writtenby:RickRiordan,Narratedby:RobbieDaymond,13 hrs and 8 mins,01-05-18,English,4.5 out of 5 stars41 ratings,820.00
2,The Deep End,Writtenby:JeffKinney,Narratedby:DanRussell,2 hrs and 3 mins,06-11-20,English,4.5 out of 5 stars38 ratings,410.00


## Cleaning Data

In [5]:
def extract_time(time_str):
    time_str = time_str.lower()
    if 'lethan1ute' in time_str or 'less than 1 minute' in time_str:
        return 0, 0.5
    hours = re.search(r'(\d+)\s*hr', time_str)
    minutes = re.search(r'(\d+)\s*min', time_str)
    hours = float(hours.group(1)) if hours else 0
    minutes = float(minutes.group(1)) if minutes else 0
    return hours, minutes


In [6]:
cleaned = df\
        .copy().assign(
            
            author=lambda x: x['author']
                .str.replace("Writtenby:", "", regex=True)
                .str.replace(r'([a-z])([A-Z])', r'\1 \2', regex=True)
                .str.split(","),

            narrator=lambda x: x['narrator']
                .str.replace("Narratedby:", "", regex=True)
                .str.replace(r'([a-z])([A-Z])', r'\1 \2', regex=True)
                .str.split(","),

            language=lambda x: x['language'].str.title(),

            releasedate=lambda x: pd.to_datetime(x['releasedate'], format='%d-%m-%y'),
            
            Rating=lambda x: x['stars']
                .str.replace(r' rating|out of 5 star|s', '', regex=True)
                .where(x['stars'] != 'Not rated yet', np.nan)
                .str.split(" ", n=2, expand=True)[0].astype(pd.Float32Dtype()),
                
            Reviews=lambda x: x['stars']
                .str.replace(r' rating|out of 5 star|s|,', '', regex=True)
                .where(x['stars'] != 'Not rated yet', np.nan)
                .str.split(" ", n=2, expand=True)[1].astype(pd.Int64Dtype()),
            
            Runtime=lambda x: ((x['time'].apply(lambda y: extract_time(y)[0])) * 60 +
                               x['time'].apply(lambda y: extract_time(y)[1])),

            
            price=lambda x: x['price']
                .str.replace(",", "")
                .str.replace("Free", "0")
                .astype(pd.Float64Dtype()),
            
            usd=lambda x: (x["price"]
                .astype(str)                       
                .str.replace(",", "")
                .str.replace("Free", "0")
                .replace("nan", "0")
                .fillna("0")
                .astype(float)
                * conversion_rate).round(2))\
        .drop(['stars', 'time'], axis=1)\
        .rename(columns={
                    "name": "Title",
                    "author": "Author",
                    "narrator": "Narrator",
                    "releasedate": "Release Date",
                    "language": "Language",
                    "price": "INR Price",
                    "usd": "USD Price"})

## Saving Data

In [7]:
cleaned.to_parquet("./Data/Clean/audible_pandas.parquet")

# Names

In [204]:
import pandas as pd
from lets_plot import *
LetsPlot.setup_html()

In [206]:
names = pd.read_csv('https://github.com/byuidatascience/data4names/raw/master/data-raw/names_year/names_year.csv')

In [234]:
Isaac = names[names['name'] == 'Kaylee']
ggplot(Isaac, aes(y = Isaac['Total'], x = Isaac['year'])) + \
    geom_line()


In [212]:
Brittnay = names[names['name'] == 'Brittnay']
ggplot(Brittnay, aes(y = Brittnay['Total'], x = Brittnay['year'])) + \
    geom_line()


In [215]:
Apostles = names[(names['name'].isin(['Peter', 'Mary', 'Martha', 'Paul'])) & names['year'].between(1920, 2000)]
ggplot(Apostles, aes(y = 'Total', x='year', color='name')) + \
    geom_line()

In [235]:
Famous = names[(names['name'].isin(['Elvis', 'Han', 'Leia']))]
(ggplot(Famous, aes(x = 'year', y = 'Total', color='name')) + 
    geom_line())

# Late Flights

In [248]:
import pandas as pd
import numpy as np
from lets_plot import *
LetsPlot.setup_html()

In [239]:
df = pd.read_json("https://github.com/byuidatascience/data4missing/raw/master/data-raw/flights_missing/flights_missing.json")

In [ ]:
df

In [275]:
df['num_of_delays_nas']#.unique()

0      4598
1       935
2       895
3      5415
4       638
       ... 
919      61
920    1364
921     256
922    2372
923     404
Name: num_of_delays_nas, Length: 924, dtype: int64

In [276]:
df = df.copy().assign(
    airport_code = lambda x: x['airport_code'],
    month = lambda x: x['month']
        .replace({'Febuary': 'Februrary',
                  'n/a': np.nan}),
    num_of_delays_carrier = lambda x: x['num_of_delays_carrier']
        .str.replace(r"+", "").astype(int),
    num_of_delays_late_aircraft = lambda x: x['num_of_delays_late_aircraft']
        .replace(-999, np.nan)
)#['num_of_delays_carrier'].unique()

In [277]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 924 entries, 0 to 923
Data columns (total 17 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   airport_code                   924 non-null    object 
 1   airport_name                   924 non-null    object 
 2   month                          897 non-null    object 
 3   year                           901 non-null    float64
 4   num_of_flights_total           924 non-null    int64  
 5   num_of_delays_carrier          924 non-null    int32  
 6   num_of_delays_late_aircraft    884 non-null    float64
 7   num_of_delays_nas              924 non-null    int64  
 8   num_of_delays_security         924 non-null    int64  
 9   num_of_delays_weather          924 non-null    int64  
 10  num_of_delays_total            924 non-null    int64  
 11  minutes_delayed_carrier        872 non-null    float64
 12  minutes_delayed_late_aircraft  924 non-null    int

# Star Wars

In [23]:
import pandas as pd 
import numpy as np
from lets_plot import *
LetsPlot.setup_html()

In [42]:
df = pd.read_csv("https://github.com/fivethirtyeight/data/raw/master/star-wars-survey/StarWars.csv", encoding='utf-8', skiprows=1)
questions = pd.read_csv("https://github.com/fivethirtyeight/data/raw/master/star-wars-survey/StarWars.csv", encoding='ISO-8859-1', skiprows=0).columns

In [203]:
df = df\
    .copy()\
    .rename(columns={"Unnamed: 0": "Id",
                          "Response": "SeenStarWars",
                          "Response.1": "StarWarsFan",
                          "Star Wars: Episode I  The Phantom Menace": "EpI",
                          "Star Wars: Episode II  Attack of the Clones": "EpII",
                          "Star Wars: Episode III  Revenge of the Sith": "EpIII",
                          "Star Wars: Episode IV  A New Hope": "EpIV",
                          "Star Wars: Episode V The Empire Strikes Back": "EpV",
                          "Star Wars: Episode VI Return of the Jedi": "EpVI",
                          "Star Wars: Episode I  The Phantom Menace.1": "RankEpI",
                          "Star Wars: Episode II  Attack of the Clones.1": "RankEpII",
                          "Star Wars: Episode III  Revenge of the Sith.1": "RankEpIII",
                          "Star Wars: Episode IV  A New Hope.1": "RankEpIV",
                          "Star Wars: Episode V The Empire Strikes Back.1": "RankEpV",
                          "Star Wars: Episode VI Return of the Jedi.1": "RankEpVI",
                          "Han Solo": "RankHan",
                          "Luke Skywalker": "RankLuke",
                          "Princess Leia Organa": "RankLeia",
                          "Anakin Skywalker": "RankAnakin",
                          "Obi Wan Kenobi": "RankObiWan",
                          "Emperor Palpatine": "RankPalp",
                          "Darth Vader": "RankVader",
                          "Lando Calrissian": "RankLando",
                          "Boba Fett": "RankBoba",
                          "C-3P0": "RankC-3PO",
                          "R2 D2": "RankR2-D2",
                          "Jar Jar Binks": "RankBinks",
                          "Padme Amidala": "RankPadme",
                          "Yoda": "RankYoda",
                          "Response.2": "ShotFirst",
                          "Response.3": "ExpandedUniverse",
                          "Response.4": "FanExpandedUniverse",
                          "Response.5": "StarTrek",
                          "Response.6": "Sex",
                          "Response.7": "AgeRange",
                          "Response.8": "IncomeRange",
                          "Response.9": "Education",
                          "Response.10": "CensusRegion"
                          })

In [179]:
final = df.assign(
        **{col: lambda x, col=col: x[col].str.startswith(r"Star|Yes").replace({True: True, False: False, np.nan: False}) 
               for col in ["ExpandedUniverse", "SeenStarWars", "StarWarsFan",
                           "EpI", "EpII", "EpIII", "EpIV", "EpV", "EpVI",
                           "FanExpandedUniverse", "StarTrek"]},
        **{col: lambda x, col=col: x[col].replace({'Very favorably': 5, 
                                                   "Somewhat favorably": 4,
                                                   "Neither favorably nor unfavorably (neutral)": 3,
                                                   "Somewhat unfavorably": 2,
                                                   "Very unfavorably": 1,
                                                   "Unfamiliar (N/A)": 0})
            for col in df.columns if col.startswith("Rank") and not col.startswith("RankEp")},
        ShotFirst=lambda x: x['ShotFirst']
                .replace({"I don't understand this question": "No Opinion",
                          np.nan: "No Idea"}),
        Sex=lambda x: x['Sex'].replace({'Male': "M",
                                        'Female': "F"}),
        
        AvgAge=lambda x: (
                (x['AgeRange'].str.replace("> 60", "60-100")
                 .fillna("0-0")  # Fill NaN with a dummy value
                 .str.split("-").str[0].astype(int)) +
                (x['AgeRange'].str.replace("> ", "-")
                 .fillna("0-0")
                 .str.split("-").str[1].astype(int))
             / 2).round(1),
        Income=lambda x: (
            (x['IncomeRange']
             .str.replace(r"$", "")
             .str.replace('+', '')
             .str.split('-').str[0].str.replace(',', '').fillna(0).astype(int) +  # Remove commas here
             x['IncomeRange']
             .str.replace("$", "")
             .str.replace('+', '')
             .str.split('-').str[1].str.replace(',', '').fillna(0).astype(int)) / 2
        ).round(0),
        Education=lambda x: (
                x['Education']
                .replace({"High school degree": "High School",
                              "Bachelor degree": "Bachelors",
                              "Some college or Associate degree": "Some College",
                              "Graduate degree": "Graduate",
                              "Less than high school degree": "No Degree"})
        ))


    #.info()

In [181]:
final_summary = final.groupby('AgeRange', as_index=False).agg({'Income': 'mean'})

custom_order = ['> 60', '18-29', '30-44', '45-60'] 
final_summary['AgeRange'] = pd.Categorical(final_summary['AgeRange'], categories=custom_order, ordered=True)

p = (ggplot(final_summary, aes(x='AgeRange', y='Income')) + 
     geom_bar(stat='identity', fill='blue', width=0.7) +
     labs(x='Age Range', y='Mean Income') )

p


In [185]:


final_summary = final.groupby('AgeRange', as_index=False).agg({'Income': 'mean'})

final_summary = final_summary.sort_values(by='Income', ascending=True)

p = (ggplot(final_summary, aes(x='AgeRange', y='Income')) + 
     geom_bar(stat='identity', fill='blue', width=0.7) +
     labs(x='Age Range', y='Mean Income'))

p


In [174]:

final_summary = final.groupby('AgeRange', as_index=False).agg({'Income': 'mean'})
p = (ggplot(final_summary, aes(x='AgeRange', y='Income')) + 
     geom_bar(stat='identity', fill='blue', width=0.7) +
     labs(x='Average Age', y='Mean Income')  # Add labels for clarity
     )  # Rotate x-axis labels if needed

p


In [202]:
import pandas as pd
from lets_plot import *


final_summary = final.groupby(['Education', 'IncomeRange']).size().reset_index(name='Count')

total_counts = final_summary.groupby('Education')['Count'].sum().reset_index(name='TotalCount')

final_summary = final_summary.merge(total_counts, on='Education')
final_summary['Proportion'] = final_summary['Count'] / final_summary['TotalCount']
custom_order = ['No Degree', 'High School', 'Some College', 'Bachelors', 'Graduate']  
final_summary['Education'] = pd.Categorical(final_summary['Education'], categories=custom_order, ordered=True)

p = (ggplot(final_summary, aes(x='Education', y='Proportion', fill='IncomeRange')) + 
     geom_bar(stat='identity', width=0.7) +
     labs(x='Age Range', y='Proportion of EpII', title='Proportion of EpII by Age Range and Income Range')) 

p
